<a href="https://www.kaggle.com/code/jobaerislam/lung-colon?scriptVersionId=239730938" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.applications import MobileNetV2
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

2025-05-14 07:01:53.627723: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747206113.652280      95 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747206113.659792      95 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
dataset_dir = '/kaggle/input/lung-and-colon-cancer-histopathological-images/lung_colon_image_set'

In [3]:
img_height, img_width = 224, 224

In [4]:
def load_data(dataset_dir):
    images = []
    labels = []

    main_folders = ['colon_image_sets', 'lung_image_sets']

    for main_folder in main_folders:
        main_path = os.path.join(dataset_dir, main_folder)
        for folder in os.listdir(main_path):
            label = folder
            folder_path = os.path.join(main_path, folder)
            for filename in os.listdir(folder_path):
                img = cv2.imread(os.path.join(folder_path, filename))
                img = cv2.resize(img, (img_height, img_width))
                images.append(img)
                labels.append(label)

    images = np.array(images)
    labels = np.array(labels)
    return images, labels

images, labels = load_data(dataset_dir)

In [5]:
label_dict = {
    'colon_aca': 0, 
    'colon_n': 1, 
    'lung_aca': 2, 
    'lung_n': 3, 
    'lung_scc': 4
}
labels = np.array([label_dict[label] for label in labels])
labels = to_categorical(labels)

In [6]:
X_train, X_temp, y_train, y_temp = train_test_split(
    images, labels, test_size=0.3, random_state=42)

X_test, X_val, y_test, y_val = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

In [7]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

2025-05-14 07:04:05.106156: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(5, activation='softmax')
])

In [9]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
early_stop = EarlyStopping(
    monitor='val_loss',
    verbose=1,
    patience=3,
    restore_best_weights=True
)

lr_reduce = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2
)

In [11]:
history =  model.fit(X_train, y_train, batch_size=32, epochs=50,
          validation_data=(X_val, y_val),
          callbacks=[early_stop, lr_reduce])

Epoch 1/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1966s 4s/step - accuracy: 0.9124 - loss: 0.2474 - val_accuracy: 0.1952 - val_loss: 71.6944 - learning_rate: 0.0010
Epoch 2/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1906s 3s/step - accuracy: 0.9754 - loss: 0.0796 - val_accuracy: 0.1979 - val_loss: 43.9074 - learning_rate: 0.0010
Epoch 3/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1895s 3s/step - accuracy: 0.9858 - loss: 0.0444 - val_accuracy: 0.5867 - val_loss: 5.0539 - learning_rate: 0.0010
Epoch 4/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1896s 3s/step - accuracy: 0.9896 - loss: 0.0349 - val_accuracy: 0.4261 - val_loss: 12.3456 - learning_rate: 0.0010
Epoch 5/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1895s 3s/step - accuracy: 0.9903 - loss: 0.0344 - val_accuracy: 0.6365 - val_loss: 2.1949 - learning_rate: 0.0010
Epoch 6/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1914s 4s/step - accuracy: 0.9922 - loss: 0.0253 - val_accuracy: 0.4477 - val_loss: 5.2129 - learning_rate: 0.0010
Epoch 7/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1913s 3s/step - accuracy: 0.9885 

In [12]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy:.4f}")

118/118 ━━━━━━━━━━━━━━━━━━━━ 67s 570ms/step - accuracy: 0.9989 - loss: 0.0047
Test Accuracy: 0.9992
